In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [ ]:
df

In [17]:
df.dtypes

datetime               datetime64[ns]
holiday                       float64
workingday                    float64
weather                        object
temp                          float64
atemp                         float64
humidity                      float64
windspeed                     float64
registered_customer           boolean
dtype: object

In [ ]:
msn.matrix(df)

In [3]:
# df = df.set_index('date')
df_slice1 = df.groupby(pd.Grouper(key="datetime",freq="D")).count()

In [ ]:
#df_slice1 = df.groupby(df.datetime.year, df.datetime.month, df.datetime.day).count()

In [4]:
df_slice1

,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
datetime,,,,,,,,
2020-01-04,985,985,985,985,985,985,985,985
2020-01-05,801,801,801,801,801,801,801,801
2020-01-06,1349,1349,1349,1349,1349,1349,1349,1349
2020-01-07,1562,1562,1562,1562,1562,1562,1562,1562
2020-01-08,1600,1600,1600,1600,1600,1600,1600,1600
...,...,...,...,...,...,...,...,...
2021-12-30,4231,4231,4231,4231,4231,4231,4231,4231
2021-12-31,8144,8144,8144,8144,8144,8144,8144,8144
2022-01-01,5932,5932,5932,5932,5932,5932,5932,5932


In [12]:
px.scatter(df_slice1, x=df_slice1.index, y="registered_customer")

Vorgegebene Aufgaben

In [5]:
df_slice2 = df[df["registered_customer"] == False].copy()

In [6]:
df_slice2

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
8,2020-01-04 00:31:22,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,False
9,2020-01-04 00:35:10,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,False
11,2020-01-04 00:42:24,0.0,0.0,"clear, few clouds",9.84,14.395,81.0,0.0000,False
17,2020-01-04 01:00:14,0.0,0.0,"clear, few clouds",9.02,13.635,80.0,0.0000,False
24,2020-01-04 01:11:21,0.0,0.0,"clear, few clouds",9.02,13.635,80.0,0.0000,False
...,...,...,...,...,...,...,...,...,...
3755333,2022-01-03 23:23:12,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,False
3755334,2022-01-03 23:23:18,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,False
3755352,2022-01-03 23:32:50,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,False
3755353,2022-01-03 23:33:34,0.0,1.0,"clear, few clouds",10.66,13.635,65.0,8.9981,False


In [7]:
profile = ProfileReport(df_slice2, title='Idk', vars={"num": {"low_categorical_threshold": 0}})

In [ ]:
profile.to_notebook_iframe()

maybe group nach stunde dann count und mean von temp und atemp
dann lineare regression

In [11]:
df_slice3 = df_slice2.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "mean", "atemp": "mean", "windspeed": "mean", "humidity": "mean",})

In [12]:
df_slice3

,registered_customer,temp,atemp,windspeed,humidity
datetime,,,,,
2020-01-04 00:00:00,3,9.84,14.395,0.0000,81.0
2020-01-04 01:00:00,8,9.02,13.635,0.0000,80.0
2020-01-04 02:00:00,5,9.02,13.635,0.0000,80.0
2020-01-04 03:00:00,3,9.84,14.395,0.0000,75.0
2020-01-04 04:00:00,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-01-03 19:00:00,9,10.66,12.880,11.0014,60.0
2022-01-03 20:00:00,5,10.66,12.880,11.0014,60.0
2022-01-03 21:00:00,4,10.66,12.880,11.0014,60.0


In [13]:
df_slice3.dropna()

,registered_customer,temp,atemp,windspeed,humidity
datetime,,,,,
2020-01-04 00:00:00,3,9.84,14.395,0.0000,81.0
2020-01-04 01:00:00,8,9.02,13.635,0.0000,80.0
2020-01-04 02:00:00,5,9.02,13.635,0.0000,80.0
2020-01-04 03:00:00,3,9.84,14.395,0.0000,75.0
2020-01-04 06:00:00,2,9.02,13.635,0.0000,80.0
...,...,...,...,...,...
2022-01-03 19:00:00,9,10.66,12.880,11.0014,60.0
2022-01-03 20:00:00,5,10.66,12.880,11.0014,60.0
2022-01-03 21:00:00,4,10.66,12.880,11.0014,60.0


In [14]:
df_slice3.corr()

,registered_customer,temp,atemp,windspeed,humidity
registered_customer,1.000000,0.114899,0.453973,0.091659,-0.356967
temp,0.114899,1.000000,0.248092,-0.002570,-0.014912
atemp,0.453973,0.248092,1.000000,-0.063259,-0.055496
windspeed,0.091659,-0.002570,-0.063259,1.000000,-0.291329
humidity,-0.356967,-0.014912,-0.055496,-0.291329,1.000000


unterschied korrelationskoeffizient: temp r= 0,11 und atemp r= 0,45


In [19]:
df_sclice4= df_slice2[(df.holiday==1)&(df.windspeed>35)].copy()

C:\Users\paulh\AppData\Local\Temp\ipykernel_17708\512203837.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_sclice4= df_slice2[(df.holiday==1)&(df.windspeed>35)].copy()


In [20]:
df_sclice4

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
929511,2020-09-07 22:04:56,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929513,2020-09-07 22:05:52,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929514,2020-09-07 22:06:09,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929518,2020-09-07 22:07:14,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929538,2020-09-07 22:21:15,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
...,...,...,...,...,...,...,...,...,...
1387221,2020-12-28 11:51:22,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387228,2020-12-28 11:53:35,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387233,2020-12-28 11:54:38,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387240,2020-12-28 11:57:49,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False


In [21]:
df_slice5 = df_sclice4.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "mean", "atemp": "mean", "windspeed": "mean", "humidity": "mean",})

In [24]:
df_slice5.dropna()

,registered_customer,temp,atemp,windspeed,humidity
datetime,,,,,
2020-09-07 22:00:00,18,24.60,27.275,39.0007,88.0
2020-11-13 10:00:00,38,13.12,13.635,36.9974,42.0
2020-11-13 13:00:00,57,15.58,19.695,36.9974,37.0
2020-12-28 10:00:00,51,446.90,14.395,35.0008,42.0
2020-12-28 11:00:00,43,13.94,14.395,39.0007,46.0


In [25]:
df_slice5.corr()

,registered_customer,temp,atemp,windspeed,humidity
registered_customer,1.000000,0.341437,-0.623625,-0.626736,-0.915651
temp,0.341437,1.000000,-0.314862,-0.793264,-0.218412
atemp,-0.623625,-0.314862,1.000000,0.485406,0.844940
windspeed,-0.626736,-0.793264,0.485406,1.000000,0.585789
humidity,-0.915651,-0.218412,0.844940,0.585789,1.000000


In [ ]:
# X = df_slice3["temp"]
# Y = df_slice3["registered_customer"]
# X.array.reshape(-1,1)
#array.reshape(-1, 1)

In [ ]:
# from sklearn import datasets, linear_model
# from sklearn.metrics import mean_squared_error, r2_score
# regr = linear_model.LinearRegression()
# regr.fit(X,Y)
# print("inputs", X.columns,"\n")
# print("Corresponding: \n", regr.coef_)
# print("Intercept: \n", regr.intercept_)